# Práctica 3: Caracterización y Clasificación de Texturas
- Martínez Ostoa Néstor Iván
- Ramírez Bondi Jorge Alejandro

## 1. Construcción de imágenes de prueba y entrenamiento

Para construir el conjunto de imágenes de prueba y entrenamiento tomamos un subconjunto de las texturas de [Brodatz](http://sipi.usc.edu/database/database.php?volume=textures), las subdividimos en $n$ ventanas de tamaños iguales y separamos en ventanas de prueba y entrenamiento.

In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
def split_image(image, num_images, verbose=False):
    """
    Splits an image (ndarray) into 'num_images' images and returns the splitted
    images

    Parameters:
    -----------
    'image': numpy ndarray
        'image' must be a squared 2D numpy array 

    'num_images': int
        indicates the number of images to be returned

    Returns:
    --------
    'splitted_images': numpy ndarray
    """
    (width, height) = image.shape
    if width != height: raise Exception('image must be a squared ndarray')
    num_windows = int(np.sqrt(num_images))
    window_size = width // num_windows
    if verbose:
        print(f"\tImage of size ({width}x{height}). Window size: {window_size}")
    splitted_images = []
    y = 0
    while (y+window_size <= height):
        x = 0
        while (x+window_size <= width):
            if verbose: print(f"\t({x},{x+window_size}:{y},{y+window_size})\t", end='')
            splitted_images.append(image[x:x+window_size, y:y+window_size])
            x += window_size
        if verbose: print()
        y += window_size
    splitted_images = np.array(splitted_images)
    return splitted_images

In [3]:
def save_images(base_path, img_name, images, prefix):
    img_name = img_name.split('.')
    base_name = img_name[0]
    extension = img_name[1]
    for idx, img in enumerate(images):
        im = Image.fromarray(img)
        im.save(base_path + prefix + '/' + base_name + '_' + str(idx) + '.' + extension)

In [4]:
textures_path = '../textures/train/'
textures_names = os.listdir(textures_path)
base_path = 'imgs/'
num_windows = 9
for texture_name in textures_names:
    if texture_name[0] == '.': continue
    print(f"Processing {texture_name}")
    # Image reading
    rgb_img = cv2.imread(textures_path + texture_name)
    gray_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)

    # Image splitting
    images = split_image(gray_img, num_windows, verbose=False)
    test_imgs, train_imgs = train_test_split(images, test_size=0.2)

    # Image saving
    save_images(base_path, texture_name, test_imgs, prefix='test')
    save_images(base_path, texture_name, train_imgs, prefix='train')

Processing Piedras.jpg
Processing D16.bmp
Processing D6.bmp
Processing D64.bmp
Processing D49.bmp
Processing D46.bmp
Processing Piedras3.jpg
Processing D101.bmp


## 2. Vectores característicos de entrenamiento
Para esta seccción utilizamos las imágenes de entrenamiento

In [5]:
import mahotas as mt

In [6]:
def get_features(img, verbose=False):
    """
    Returns the 13 statistics defined by Haralick for each image that is passed
    as input:
        - Angular second moment
        - Contrast
        - Correlation
        - Variance
        - Inverse Difference Moment
        - Sum Average
        - Sum Variance
        - Sum Entropy
        - Entropy
        - Difference Variance
        - Difference Entropy
        - Info Measure of Correlation 1
        - Info Measure of Correlation 2

    Parameters:
    -----------
    'img': ndarray

    Returns:
    --------
    'feature_vector': ndarray
        each element inside 'feature_vector' represents one of the 13th Haralick 
        statistics. By using mean(0) we are getting the mean of the 13 Haralick
        statistics per co-ocurrence matrix direction. Otherwise, 'feature_vector'
        would have 4x13 dimensions (one per matrix direction)
    """
    feature_vector = mt.features.haralick(img).mean(0) # Dimensions: (1x13)
    return feature_vector

In [7]:
train_features = []
train_labels = []

path = 'imgs/train/'
imgs_names = os.listdir(path)
for img_name in imgs_names:
    if img_name[0] == '.': continue
    class_name = img_name.split('_')[0]
    print(f"Processing {class_name}")
    # Image reading
    img = cv2.cvtColor(cv2.imread(path + img_name), cv2.COLOR_BGR2GRAY)

    # Feature extraction
    img_features = get_features(img, verbose=True)
    train_features.append(img_features)
    train_labels.append(class_name)

train_features = np.array(train_features)
train_labels = np.array(train_labels)
print(f"\n-----\nNumber of features: {train_features.shape}")
print(f"Number of labels: {train_labels.shape}")

Processing D46
Processing D46
Processing D64
Processing Piedras3
Processing Piedras3
Processing D64
Processing D6
Processing D6
Processing D16
Processing D16
Processing Piedras
Processing Piedras
Processing D49
Processing D101
Processing D101
Processing D49

-----
Number of features: (16, 13)
Number of labels: (16,)


In [8]:
train_labels

array(['D46', 'D46', 'D64', 'Piedras3', 'Piedras3', 'D64', 'D6', 'D6',
       'D16', 'D16', 'Piedras', 'Piedras', 'D49', 'D101', 'D101', 'D49'],
      dtype='<U8')

## 3. Clasificador

Para la clasificación, utilizamos dos clasificadores distintos:

1. Máquina de soporte vectorial (SVM)
2. KNN

In [9]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

### Entrenamiento

In [10]:
svm = LinearSVC(random_state=10, max_iter=1e8)
svm.fit(train_features, train_labels)

/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(max_iter=100000000.0, random_state=10)

In [11]:
knn = KNeighborsClassifier()
knn.fit(train_features, train_labels)

KNeighborsClassifier()

### Predicciones y Validación

In [12]:
def get_unique_features(imgs_names):
    features = set()
    for name in imgs_names:
        class_name = name.split('_')[0]
        features.add(class_name)
    return np.array(list(features))

def num_feature_from_string(unique_features, str_feature):
    idx = 0
    for f in unique_features:
        if f == str_feature: return idx
        idx += 1
    return -1

In [13]:
svm_pred = []
knn_pred = []
real_features = []

path = 'imgs/test/'
imgs_names = os.listdir(path)
unique_features = get_unique_features(imgs_names[1:])
for idx, img_name in enumerate(imgs_names):
    if img_name[0] == '.': continue
    class_name = img_name.split('_')[0]
    real_features.append(num_feature_from_string(unique_features, class_name))
    
    # Image reading
    img = cv2.cvtColor(cv2.imread(path + img_name), cv2.COLOR_BGR2GRAY)

    # Feature extraction
    features = get_features(img)

    # SVM prediction
    svm_prediction = svm.predict(features.reshape(1, -1))[0]
    svm_pred.append(num_feature_from_string(unique_features, svm_prediction))

    # KNN prediction
    knn_prediction = knn.predict(features.reshape(1, -1))[0]
    knn_pred.append(num_feature_from_string(unique_features, knn_prediction))

    #if idx < 10:
        # Image showing
     #   cv2.putText(img, knn_prediction, (20,30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,0,0))
      #  cv2.imshow('Test image', img)
       # cv2.waitKey(0)

### Evaluación entre SVM y KNN

Para evaluar el desempeño de la clasificación entre SVM y KNN, utilizamos **f1-score** pues es una métrica que combina la precisión y *recall*. Por un lado, la precisión es una métrica que nos ayuda a minimizar el número de falsos positivos. Por otro lado, *recall* es una métrica que es importante en experimentos de índole médica, por ejemplo, pues queremos minimizar la posibilidad de que se nos escapen los casos positivos. Por esta razón, **f1-score** es la mejor métrica pues combina estas dos métricas en una sola. 

In [14]:
from sklearn.metrics import f1_score

In [15]:
svm_f1 = f1_score(real_features, svm_pred, average='macro')
knn_f1 = f1_score(real_features, knn_pred, average='macro')
print(f'F1 score for SVM: {svm_f1}')
print(f'F1 score for KNN: {knn_f1}')

F1 score for SVM: 0.2885416666666667
F1 score for KNN: 0.45208333333333334


## 4. Evaluación sobre imágenes compuestas

In [18]:
path = 'imgs/mixed/'
imgs_names = os.listdir(path)
for idx, img_name in enumerate(imgs_names):
    if img_name[0] == '.': continue
    class_name = img_name.split('_')[0]
    print(class_name)
    real_features.append(num_feature_from_string(unique_features, class_name))

    # Image reading
    img = cv2.cvtColor(cv2.imread(path + img_name), cv2.COLOR_BGR2GRAY)

    # Image splitting (four windows)
    images = split_image(img, num_images=4, verbose=False)

    # Image showing
    for img_idx, img in enumerate(images):
        features = get_features(img)
        prediction = knn.predict(features.reshape(1, -1))[0]
        print(f"\tPrediction #{img_idx + 1}: {prediction}")
        #cv2.imshow('Test', img)
        #cv2.waitKey(0)



imgCompuesta2.png
	Prediction #1: D16
	Prediction #2: D46
	Prediction #3: D46
	Prediction #4: D101
imgCompuesta3.png
	Prediction #1: D101
	Prediction #2: D46
	Prediction #3: D46
	Prediction #4: D46
imgCompuesta1.png
	Prediction #1: D16
	Prediction #2: D49
	Prediction #3: D46
	Prediction #4: D101


In [ ]:
unique_features